![](http://)![](http://)Importing the required library

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from seaborn import countplot,lineplot, barplot
import math
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense
from mlxtend.classifier import SoftmaxRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from numpy import argmax
from sklearn.metrics import accuracy_score
from keras.regularizers import l1
from scipy import stats
from keras.layers import Dropout
import statistics
import os
import random
print(os.listdir("../input"))

**Read the Data**

In [ ]:
xtest = pd.read_csv("../input/X_test.csv")
xtrain = pd.read_csv("../input/X_train.csv")
ytrain = pd.read_csv("../input/y_train.csv")
ssb = pd.read_csv("../input/sample_submission.csv")

![](http://)![](http://)**Check the shape of imported data**

In [ ]:
xtest.shape, xtrain.shape, ytrain.shape, ssb.shape

**Check the different type of floor present**

In [ ]:
print('Number of surface: {}'.format(ytrain.surface.nunique()))
countplot(y = 'surface', data = ytrain)
plt.show()

From the chart we found that hard_tiles has very less number of dataset which might effect out our model prediction on hard_tiles observation. To develop better model it suggested to gather more data for Hard_tile floor. On the other hand cocrete contains the maximum dataset which results better prediction on concrete. As the difference between the dataset of concrete and hard_tile is very high it can lead us under fitting the resul on hard_tile and overfitting the result on concrete 

**Check our imported dataset**

In [ ]:
xtrain.head()

**Check the shape of our test set result**

In [ ]:
ytrain.shape

**Check our imported test data**

In [ ]:
xtest.head()

As we know robot is free to move in all 3 direction and final movement velocity depends on resultant velocity of robot. 
W = Wx+Wy+Wz
To increase the accuracy of our model first we will list down all the required parameter that will affect robot movement

In [ ]:
resultant_velocity = (xtrain['angular_velocity_X']**2+
xtrain['angular_velocity_Y']**2+  
xtrain['angular_velocity_Z']**2)**.5
resultant_velocity= pd.DataFrame(resultant_velocity, columns = ['resultant_velocity'])
#sc_x = StandardScaler()
#resultant_velocity_n = pd.DataFrame(sc_x.fit_transform(resultant_velocity),columns=resultant_velocity.columns, index=resultant_velocity.index )
#resultant_velocity_n = np.array(resultant_velocity_n)
resultant_velocity.shape
resultant_velocity.head()

Resultant acceleration = sq.root(Ax^2+Ay^2+Az^2)

In [ ]:
resultant_acc = (xtrain['linear_acceleration_X']**2+
xtrain['linear_acceleration_Y']**2+  
xtrain['linear_acceleration_Z']**2)**.5
#resultant_acc= np.transpose(np.matrix(np.array(resultant_acc.T)))
resultant_acc= pd.DataFrame(resultant_acc, columns = ['resultant_acc'])
#sc_y = StandardScaler()
#resultant_acc = pd.DataFrame(sc_x.fit_transform(resultant_acc),columns=resultant_acc.columns, index=resultant_acc.index )
resultant_acc.head()

For robot, Difference between different floor is fiction provided by different floor. All 9 floor mentioned in problem statement has different frictional resistance.

**Friction:**  
• The resistance between two surfaces when attempting to slide one object across the other.  
• Friction is due to interactions at molecular level where “rough edges” bond together:
• Friction is always opposite to the direction of motion
As the robot overcome the friction force by its internal power. Power is a fuction of mass, acceleration and velocity of moving object. Hence we consider power as one of factor for our model




In [ ]:
power=resultant_velocity['resultant_velocity']*resultant_acc['resultant_acc']
#As floor is fuction of friction, To estimate the friction factor differ force eq is required. Power is one of the factor on which friction force depends
power= pd.DataFrame(power, columns = ['power'])
power.head()
#xtrain_new = np.hstack((xtrain,resultant_velocity,resultant_acc,power))

In [ ]:
#x, y, z, w = xtrain['orientation_X'].tolist(), xtrain['orientation_Y'].tolist(), xtrain['orientation_Z'].tolist(), xtrain['orientation_W'].tolist()
#t0 = 2*np.multiply(x,y)

In [ ]:
#x = xtrain.iloc[:,3].values
#y = xtrain.iloc[:,4].values
#z = xtrain.iloc[:,5].values
#w = xtrain.iloc[:,6].values

#a0 = +2.0 * (np.multiply(w,x) + np.multiply(y , z))
#a1 = +1.0 - 2.0 * (np.multiply(x , x) + np.multiply(y , y))
#A = np.arctan2(a0,a1)

#a2 = +2.0 * (np.multiply(w , y) - np.multiply(z , x))
#B = np.arcsin(a2)

#a3 = +2.0 * (np.multiply(w , z) + np.multiply(x , y))
#a4 = +1.0 - 2.0 * (np.multiply(y , y) + np.multiply(z , z))
#C = np.arctan2(a3, a4)


In [ ]:
#A.shape, B.shape, C.shape

In [ ]:
#xtrain['euler_rotation_x'] = A
#xtrain['euler_rotation_y'] = B
#xtrain['euler_rotation_z'] = C
#xtrain['resultant_angle'] = (xtrain['euler_rotation_x'] ** 2 + xtrain['euler_rotation_y'] ** 2 + xtrain['euler_rotation_z'] ** 2)** .5

**Combine calculated parameter in a matrix**

In [ ]:
xtrain_new= pd.concat([xtrain,resultant_velocity, resultant_acc, power], axis=1)
xtrain_new.shape
xtrain_new.head()

From the data set we found that train sample result is given on the basis of series_id. To increase the training sample we expand our result as per out training dataset

In [ ]:
merged = xtrain_new.merge(ytrain, on='series_id')
merged.shape

In [ ]:
merged.head()

Extracting type of floor from our created matrix

In [ ]:
y =merged.iloc[:,16:18].values

In [ ]:
#y = pd.DataFrame(y)
#y.head()

**Creating labelencoder matrix of our outcomes of training sample**

In [ ]:
labelencoder = LabelEncoder()
y[:,1] = labelencoder.fit_transform(y[:, 1])
#y.shape
onehotencoder = OneHotEncoder(categorical_features = [1])
y = onehotencoder.fit_transform(y).toarray()
#y = onehotencoder.fit_transform(ytrain).toarray()
#Y_Train_f = Y_Train_f[:, 1:]

In [ ]:
y_train = y[:,0:9]
y_train.shape
y_Train = pd.DataFrame(y_train)
y_Train.head()

In [ ]:
y_train.shape

In [ ]:
x = xtrain_new.iloc[:,3:16]
x.shape

In [ ]:
X = pd.DataFrame(x)
X.head()

**Normalising our generated Training data set**

In [ ]:
sc = StandardScaler()
x = sc.fit_transform(x)
x=pd.DataFrame(x)
x.head()

In [ ]:
ytrain_new = xtrain_new.groupby('series_id')['power','resultant_velocity','resultant_acc'].mean()
ytrain_new = pd.DataFrame(ytrain_new).reset_index()
ytrain_new.columns = ['serie_id','avg_power','avg_velocity','avg_acc']
ytrain_new['surface'] = ytrain.surface
ytrain_new['group_id'] = ytrain.group_id

To select our model, we will first start obseving relaion between our different inputs.
Check the behaviour of power w.r.t different surface

In [ ]:
f, axes = plt.subplots(figsize=(10, 5))
sns.boxplot(x='surface',y='avg_power',data=ytrain_new, ax=axes)
plt.title('avg_power vs surface')

From the plot we found that its difficult to distinguish between different surface on the basis of power, similary we will see the relation between different independent variable


In [ ]:
f, axes = plt.subplots(figsize=(10, 5))
sns.boxplot(x='surface',y='avg_acc',data=ytrain_new, ax=axes)
plt.title('avg_acc vs surface')

In [ ]:
# Plot power vs velocity
f, axes = plt.subplots(figsize=(10, 5))
sns.lineplot(data=ytrain_new, x='avg_acc', y='avg_power', hue='surface',ax=axes)
plt.show()

In [ ]:
f, axes = plt.subplots(figsize=(10, 5))
sns.lineplot(data=ytrain_new, x='avg_velocity', y='avg_power', hue='surface',ax=axes)

In [ ]:
f, axes = plt.subplots(figsize=(10, 5))
sns.lineplot(data=ytrain_new, x='avg_velocity', y='avg_acc', hue='surface',ax=axes)

In [ ]:
f, axes = plt.subplots(figsize=(10, 5))
sns.boxplot(x='surface',y='avg_power',data=ytrain_new, ax=axes)
plt.title('avg_power vs surface')

As we see its not possible to identify the surface from any single parameter.
We will use all the identified parameter and built a ANN model to identify the surface

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y_train, test_size = 0.3, random_state = 0)

We spilt our test into training and test in the ratio of 60 to 40. To avoid under fitting problem we divide our data set (60:40). As we need more test set to verify our result

In [ ]:
Y_Test = pd.DataFrame(Y_test)
Y_Test.head()

In [ ]:
Y_TEST = Y_Test.idxmax(axis=1)
Y_TEST.head()

In [ ]:
random.seed(30)

We built our model with folowing confrigation-

Hidden layer = 4

By heat and trial method we founnd that maximum for accuracy occured with folowing activation function

First Layer - relu
Second layer - tanh
Third Layer - tanh

As we have multiple outputs (9 surfaces) we used the softmax function in the output layer

We need an optimize epoch size, if epoch size is less then it create underfitting problem and if epoch size is high it consume high processing power and create overfitting problem.
reverse is valid as for batch size

In [ ]:
    #initializing ANN
classifier = Sequential()

    # Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 20, init = 'uniform', activation = 'relu', input_dim = 13 ))

    # Adding the second hidden layer
classifier.add(Dense(output_dim = 20, init = 'uniform', activation = 'tanh',))

#classifier.add(Dropout(0.2))

    # Adding the third hidden layer
classifier.add(Dense(output_dim = 20, init = 'uniform', activation = 'tanh'))

#classifier.add(Dropout(0.2))

    # Adding the output layer
classifier.add(Dense(output_dim = 9, init = 'uniform', activation = 'softmax'))

    # Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
classifier.fit(X_train, Y_train, batch_size = 50, nb_epoch = 25)

**Predict our result over created test sample**

In [ ]:
y_pred = classifier.predict(X_test, verbose=True)
y_pred =pd.DataFrame(y_pred)

In [ ]:
y_Pred = y_pred.idxmax(axis=1)
y_Pred.head()

In [ ]:
print("Accuracy in Test set=",accuracy_score(Y_TEST, y_Pred))

**Creating dataset for to predict result on test set**

In [ ]:
resultant_velocity1 = (xtest['angular_velocity_X']**2+ xtest['angular_velocity_Y']**2+ xtest['angular_velocity_Z']**2)**.5
resultant_velocity1= pd.DataFrame(resultant_velocity1, columns = ['resultant_velocity1'])
resultant_acc1 = (xtest['linear_acceleration_X']**2+
xtest['linear_acceleration_Y']**2+  
xtest['linear_acceleration_Z']**2)**.5
#resultant_acc= np.transpose(np.matrix(np.array(resultant_acc.T)))
resultant_acc1= pd.DataFrame(resultant_acc1, columns = ['resultant_acc1'])
power1=resultant_velocity1['resultant_velocity1']*resultant_acc1['resultant_acc1']
#As floor is fuction of friction, To estimate the friction factor differ force eq is required. Power is one of the factor on which friction force depends
power1= pd.DataFrame(power1, columns = ['power1'])


In [ ]:
resultant_velocity1.head()

In [ ]:
resultant_acc1.head()

In [ ]:
power1.head()

In [ ]:
#xt = xtest.iloc[:,3].values
#yt = xtest.iloc[:,4].values
#zt = xtest.iloc[:,5].values
#wt = xtest.iloc[:,6].values

#b0 = +2.0 * (np.multiply(wt,xt) + np.multiply(yt , zt))
#b1 = +1.0 - 2.0 * (np.multiply(xt , xt) + np.multiply(yt , yt))
#b1.shape
#Xt = np.arctan2(b0,b1)

#b2 = +2.0 * (np.multiply(wt , yt) - np.multiply(zt , xt))

#Yt = np.arcsin(b2)

#b3 = +2.0 * (np.multiply(wt , zt) + np.multiply(xt , yt))
#b4 = +1.0 - 2.0 * (np.multiply(yt , yt) + np.multiply(zt , zt))
#Zt = np.arctan2(b3, b4)

In [ ]:
#Xt.shape, Yt.shape, Zt.shape

In [ ]:
#xtest['euler_rotation_x'] = Xt
#xtest['euler_rotation_y'] = Yt
#xtest['euler_rotation_z'] = Zt  
#xtest['resultant_angle'] = (xtest['euler_rotation_x'] ** 2 + xtest['euler_rotation_y'] ** 2 + xtest['euler_rotation_z'] ** 2) ** .5

In [ ]:
xtest_new= pd.concat([xtest,resultant_velocity1, resultant_acc1, power1], axis=1)
x3 = xtest_new
xtest_new.head()



In [ ]:
x3.head()

In [ ]:
xtest_new = xtest_new.iloc[:, 3:16]
xtest_new = pd.DataFrame(xtest_new)
xtest_new.head()

In [ ]:
xtest_new.shape

In [ ]:
#xtest_new = xtest_new.groupby('series_id')['orientation_X','orientation_Y','orientation_Z','orientation_W', 'linear_acceleration_X','linear_acceleration_Y','linear_acceleration_Z', 'angular_velocity_X','angular_velocity_Y','angular_velocity_Z','resultant_velocity1','resultant_acc1','power1',].mean()
#xtest_new.head()
#xtest_new.shape

In [ ]:
sc = StandardScaler()
xtest_new = sc.fit_transform(xtest_new)
xtest_new= pd.DataFrame(xtest_new)
xtest_new.head()


In [ ]:
Y_test_final = classifier.predict(xtest_new)
Y_test_final=pd.DataFrame(Y_test_final)
Y_test_final.head()

In [ ]:
Y_test_final.shape

In [ ]:
Y_test_final.columns = ['carpet','concrete','fine_concrete','hard_tiles', 'hard_tiles_large_space', 'soft_pvc', 'soft_tiles','tiled','wood']

In [ ]:
Y_test_final.head()

In [ ]:
Y_test_final.shape

In [ ]:
Y_test_final = Y_test_final.idxmax(axis=1)
Y_test_final.head()

In [ ]:
Y_test_final= pd.DataFrame(Y_test_final, columns = ['surface'])
Y_test_final.head()

In [ ]:
seriesid_f = x3.iloc[:,1]
seriesid_f.head()

In [ ]:
seriesid_f= pd.DataFrame(seriesid_f, columns = ['series_id'])
seriesid_f.head()

In [ ]:
Y_test_final1 = pd.concat([seriesid_f, Y_test_final], axis=1)
Y_test_final1.head()

In [ ]:
Y_test_final1.shape

In [ ]:
#Y_test_final3 = Y_test_final1.pivot_table(values=["surface"], index=["series_id"], aggfunc=pd.mode)
Y_test_final3 = Y_test_final1.pivot_table(values=["surface"],
                                   index=["series_id"],
                                   aggfunc=lambda x: x.mode().iat[0])


In [ ]:
Y_test_final3.shape

In [ ]:
Y_test_final3.head()

In [ ]:
#Y_test_final4= Y_test_final3.iloc[:,0]


In [ ]:
#Y_test_final2 = Y_test_final1.groupby('series_id')['surface'].mode
#Y_test_final2.head()

In [ ]:
#Y_test_final = Y_test_final.iloc[:, [1,17,18,19,20,21,22,23,24,25] ].values

In [ ]:
#Y_test_final.head()

In [ ]:
#Y_test_final = Y_test_final.groupby('series_id')['surface'].mode()


In [ ]:
#Y_test_final.columns = ['carpet','concrete','fine_concrete','hard_tiles', 'hard_tiles_large_space', 'soft_pvc', 'soft_tiles','tiled','wood']

In [ ]:
#Y_test_final = Y_test_final.idxmax(axis=1)
#Y_test_final.head()

In [ ]:
#Y_test_final = pd.DataFrame(Y_test_final, columns = ['surface'])
#Y_test_final = Y_test_final.reset_index()
#Y_test_final.columns[0] = 'series_id'
#Y_test_final['series_id'] = Y_test_final.index
#Y_test_final.head()

In [ ]:
Y_test_final3.to_csv("prediction.csv", index = True, index_label = 'series_id')

**Results** - 

***Accuracy on Train dataset = 96%***

***Accuracy on Test dataset = 86%***
